## Furnace process dashboard

The main purpose of creating this dashboard is online monitoring of the furnace condition and fault detection  
Building this report based on data from the database, which aggregates data from different systems in the furnace infrastructure.  
I use `psycops2` library and SQL to get data needed from the database.  
For the dashboard creation I use `plotly` and `dash` libraries, which give enough opportunities for such tasks.

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt

from dash import Input, Output, Dash, html, dcc
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import psycopg2
from psycopg2.extras import DictCursor, RealDictCursor

pd.set_option ('display.max_column', None)

loading additional mapping data

In [2]:
failreason_map = pd.read_csv('fail_reason_map.csv', encoding='cp1251', sep=';')
failreason_map

,group_num,group_str,Name
0,4,Асимметрия между электродами,Перегруз (дисбаланс) (Избыток восстановителя)
1,4,Асимметрия между электродами,Перегруз (дисбаланс) (Недостаток восстановителя)
2,8,Дефицит шихты,Дефицит шихты
3,8,Дефицит шихты,Отсутствие шихты
4,7,Неготовность инфраструктуры,Аварийное закрытие лётки
...,...,...,...
112,5,Сбой работы газоочистки,Отсутствие подсводового пространства (Остановк...
113,5,Сбой работы газоочистки,Пробивание подсводового пространства (Остановк...
114,5,Сбой работы газоочистки,Снижение мощности ГО ниже минимально допустимо...
115,5,Сбой работы газоочистки,Сработала защита по газу в ПСН


writting the class to work with the database

In [3]:
class DB_Query:
    def __init__(self, dbname='XXXXXX', user='XXXXXXXX',
                 password='XXXXXXXX', host='XXXXXXXXXXXX'):
        self.dbname = dbname
        self.user = user
        self.password = password
        self.host = host

    def query(self, q_str, vars=None):
        try:
            conn = psycopg2.connect(dbname=self.dbname, user=self.user, 
                        password=self.password, host=self.host)
            cursor = conn.cursor(cursor_factory=RealDictCursor)
            cursor.execute(q_str, vars)
            df = cursor.fetchall()
            df = pd.DataFrame(df)
            return df
            
        except(Exception, Error) as error:
            print("Ошибка при работе с PostgreSQL", error)

        finally:
            if conn:
                cursor.close()
                conn.close()
           

In [4]:
class FailPeriods:
    
    def transform_all(self, df, col_names):
        total_df = pd.DataFrame()
        for elem in col_names:
            total_df = pd.concat([total_df, self.transform(df, elem)])
        total_df = total_df.reset_index().rename(columns={'index': 'type_order'})
        print(f'All columns processed successfully, total df shape: {total_df.shape}')
        return total_df
            
    def transform(self, df, col_name):
        print(f'Processing column {col_name}')
        df['borders'] = df[col_name].rolling(window=2).apply(self.__get_borders)
        df = df[['borders', 'timestamp']].dropna(how='any')
        if df.empty:
            print('Column processed, df is empty!!!')
            return
        else:
            df = df.pivot(columns='borders', values='timestamp')
            df.columns = ['begin', 'end']
            df['end'] = df['end'].shift(-1)
            df = df.dropna(how='any') #[~df['Begin'].isna()]
            df = df.reset_index(drop=True)
            df['failure_type'] = col_name
            df['duration'] = ((df['end'] - df['begin']).dt.seconds/60).astype('int64')
            print(f'Column processed, df shape: {df.shape}')
            return df
        
    def __get_borders(self, df):
        if list(df) == [0, 1]:
            res = 1
        elif list(df) == [1, 0]:
            res = 2
        else:
            res = np.NaN
        return res

defining lists and dicts for the *drop-down dashboard lists*

In [5]:
j_fails = ['1 - Техостанов печи', '2 - Облом электрода', '4 - Асимметрия электродов', '5 - Сбой газоочистки', '6 - Утечка воды', '7 - Неготовность инфраструктуры', '8 - Дефицит шихты', '9 - Потеря нагрузки на выпуске', '99 - Прочее']

In [6]:
m_fails = ['1 - Простой по мощности', '2 - Признаки облома', '3 - Признаки отжёга',
                '4 - Признаки асимметрии', '5 - Признаки сбоев работы газодувок',
                '6 - Признаки  течи', '7 - Признаки неготовности инфраструктуры']              

In [63]:
thresholds_dict = {'win_len': 10, 'i_ab_thres': 60, 'i_bc_thres': 60, 'i_ac_thres': 60,
                   'perem_ab_thres': 500, 'perem_bc_thres': 500, 'perem_ac_thres': 500, 'i_max': 140}

In [8]:
features_list = ['perem_ab_flag', 'perem_bc_flag', 'perem_ac_flag',
                 'perem_ab_rolled', 'perem_bc_rolled', 'perem_ac_rolled',
                 'perem_ab', 'perem_bc', 'perem_ac',
                 'i_a_max_flag', 'i_b_max_flag', 'i_c_max_flag',
                'i_ab_flag', 'i_bc_flag', 'i_ac_flag',
                'i_ab_rolled', 'i_bc_rolled', 'i_ac_rolled', 'i_ab', 'i_bc', 'i_ac', 'i_a_val0',
                 'i_b_val0', 'i_c_val0', 'w_akt_val0', 'minstralw_el1_val0', 'minstralw_el2_val0',
                 'minstralw_el3_val0', 'perem_a_val0', 'perem_b_val0', 'perem_c_val0',	'minstralpsn_1_val0',
                 'minstralpsn_2_val0', 'minstralpsn_3_val0', 'u_a_val0', 'u_b_val0', 'u_c_val0', 'tmantel1_val0',
                 'tmantel2_val0', 'tmantel3_val', 'razr_nakl_b1_val0', 'razr_nakl_b2_val0',
                 'p_a_val0', 'p_b_val0', 'p_c_val0', 'gosco2_cl_gas_val0', 'gosp_posle_gd_val0', 'gosrazr_pered_gd_val0',
                 'gosh2_cl_gas_val0', 'goso2_cl_gas_val0',	'h2_a_val0', 'h2_b_val0', 'h2_c_val0',
                 'davl_venturi_b1_val0', 'davl_venturi_b2_val0',	'cos_f', 'pakt', 'q', 'r_el1', 'r_el2',	'r_el3']

In [9]:
failures_list = ['1 - Техостанов', '2 - Облом эл-да',
                 '4 - Асимметрия', '5 - Сбой газоочистки',
                 '6 - Утечка воды', '7_50 - Негот-ть инфр-ры, W<50%',
                 '7_75 - Негот-ть инфр-ры, W<75%', '7_90 - Негот-ть инфр-ры, W<90%',
                 '7_100 - Негот-ть инфр-ры, W>=90%',
                '8 - Дефицит шихты', '9 - Потеря нагрузки',
                '99 - Прочее']

In [10]:
class RolledDifference:
    def get(self, df, feature_name, feat1, feat2, win_len, thres, abs=True):
        if abs:
            df[feature_name] = (df[feat1] - df[feat2]).abs()
        else:
            df[feature_name] = (df[feat1] - df[feat2])
            
        df[f'{feature_name}_rolled'] = df[feature_name].rolling(win_len).mean()

        df[f'{feature_name}_flag'] = df[f'{feature_name}_rolled'] > thres


writing a web application and run it

In [64]:
app = Dash('Furnace monitoring') 

db = DB_Query()

thresholds = []
for thres, value in thresholds_dict.items():
    thresholds.append(html.H6(children=f'{thres}', style={'text-align': 'left', 'margin': '0px'}))
    thresholds.append(dcc.Input(value=value, id=f'{thres}', type="number", debounce=True, placeholder=f'{thres}',
                                name = f'{thres}', style={'box-sizing': 'border-box', 'width': '100%', 'margin-bottom': 10}))
#print(thresholds)    


app.layout = html.Div([
    html.H3("Furnace monitoring",
            style={'margin-bottom': '10px', 'text-align': 'center',
                   'padding': 0, 'margin': 0}),
   
    ######################
    html.Div([
        
         html.Div([
            html.H6("Period", style={'margin': '0px', 'text-align': 'left'}),
             
            dcc.DatePickerRange(id='date_picker',  min_date_allowed=dt.date(2023, 8, 1),
                                max_date_allowed=dt.date(2023, 10, 30),
                                initial_visible_month=dt.date(2023, 10, 1),
                                start_date=dt.date(2023, 10, 17),
                                end_date=dt.date(2023, 10, 19)
                                )
            ], style={'display': 'inline-block', 'width': '39%'}),
        
        html.Div([
            html.H6(children="Features list 1",  
                    style={'text-align': 'center', 'margin': '0px'}),
            dcc.Dropdown(
                options=features_list,
                value='w_akt_val0',
                #optionHeight=25,
                placeholder='Choose the feature',
                id='feature1', style={'box-sizing': 'border-box', 'display': 'list-item'}
                #width': '40%','border-style': 'dotted', 'display': 'inline-block',
                )
            ], style={'display': 'inline-block', 'width': '20%'}),
        
        html.Div([
            html.H6(children="Features list 2",
                    style={'text-align': 'center', 'margin': '0px'}),
            dcc.Dropdown(
                options=features_list,
                value='i_a_val0',
                #optionHeight=25,
                placeholder='Choose the feature',
                id='feature2', style={'box-sizing': 'border-box', 'display': 'list-item'}
                #width': '40%','border-style': 'dotted', 'display': 'inline-block',
                )
            ], style={'display': 'inline-block', 'width': '20%'}),

        html.Div([
            html.H6(children="Features list 3",
                    style={'text-align': 'center', 'margin': '0px'}),
            dcc.Dropdown(
                options=features_list,
                value='u_a_val0',
                #optionHeight=25,
                placeholder='Choose the feature',
                id='feature3', style={'box-sizing': 'border-box', 'display': 'list-item'}
                #width': '40%','border-style': 'dotted', 'display': 'inline-block',
                )
            ], style={'display': 'inline-block', 'width': '20%'})
        
        
        ], style={'margin-top': '20px', 'padding-top': '0px'}),
    #################################################
    html.Div([
        
        html.Div(thresholds, style={'vertical-align': 'top', 'padding-top': 20, 'display': 'inline-block', 'width': '7%'}),

        html.Div([
            dcc.Graph(id='chart1', style={'box-sizing': 'border-box', 'height': 1600, 'padding':0, 'margin': 0})
            ], style={'display': 'inline-block', 'width': '92%', 'padding': 0, 'margin': 0})
        ], style={'padding': 0, 'margin': 0})
    #################################################
    
], style={'padding': 0, 'margin': 0}) 

##########################################################################################################################
@app.callback(
    Output('chart1', 'figure'),
    Input('date_picker', 'start_date'),
    Input('date_picker', 'end_date'),
    Input('feature1', 'value'),
    Input('feature2', 'value'),
    Input('feature3', 'value'),
    [Input(f'{thres}', 'value') for thres in thresholds_dict.keys()]
    
)
def update_fig1(start_date, end_date, feature1, feature2, feature3, *thres):
    ####################
    def common_chart(feature1, feature2, feature3):
        fig = make_subplots(rows=9, cols=1, shared_xaxes=True,
                            vertical_spacing=0.03, row_heights=[0.09, 0.10, 0.10, 0.10, 0.15, 0.10, 0.15, 0.11, 0.1])
    
        #######
    # melts
        fig.add_traces(px.timeline(melts_df, x_start='p_begin', x_end='p_end', y = 'y_order',
                                  color_discrete_sequence=['grey'],
                                  text='p_id')['data'], rows=1, cols=1)
        
        fig.add_traces(px.timeline(melts_df, x_start='p_s2_begin', x_end='p_s2_end', y = 'y_order',
                                  color_discrete_sequence=['green'])['data'], rows=1, cols=1)

        
        fig.add_annotation(xanchor='auto', xref="x domain", yanchor='top', yref="y domain", showarrow=False,
                           y=1.15, text='Melts', row=1, col=1)
        
        #######
    # feature plots
        
        fig.add_trace(go.Scatter(x=features_df['timestamp'], y=features_df[feature1], name=str(feature1), marker=dict(color='brown')),
                       row=2, col=1)
        fig.add_annotation(xanchor='auto', xref="x domain", yanchor='top', yref="y domain", showarrow=False,
                           y=1.15, text=str(feature1), row=2, col=1)

        fig.add_trace(go.Scatter(x=features_df['timestamp'], y=features_df[feature2], name=feature2, marker=dict(color='green')),
                       row=3, col=1)
        fig.add_annotation(xanchor='auto', xref="x domain", yanchor='top', yref="y domain", showarrow=False,
                           y=1.15, text=str(feature2), row=3, col=1)

        fig.add_trace(go.Scatter(x=features_df['timestamp'], y=features_df[feature3], name=feature3, marker=dict(color='grey')),
                       row=4, col=1)
        fig.add_annotation(xanchor='auto', xref="x domain", yanchor='top', yref="y domain", showarrow=False,
                           y=1.15, text=str(feature3), row=4, col=1)
        
        #######
    # asymmetry flags
        fig.add_traces(px.timeline(infrastr, x_start='begin', x_end='end',
                       y='failure_type', color='failure_type', title='Infrastructure fail flags')['data'], rows=5, cols=1)
        fig.add_annotation(xanchor='auto', xref="x domain", yanchor='top', yref="y domain", showarrow=False,
                           y=1.1, text='Asymmetry flags', row=5, col=1)

        #######
    # model_failures
        fig.add_traces(px.timeline(failures_df, x_start='dt_begin', x_end='dt_end',
                       y='event_type', color='event_type', title='Model failures')['data'], rows=6, cols=1)
        fig.add_annotation(xanchor='auto', xref="x domain", yanchor='top', yref="y domain", showarrow=False,
                           y=1.1, text='Model failures', row=6, col=1)
        
        #######
    # control flags
        
        fig.add_traces(px.timeline(control_flags, x_start='begin', x_end='end',
                       y='failure_type', color='failure_type', title='Control flags')['data'], rows=7, cols=1)
        fig.add_annotation(xanchor='auto', xref="x domain", yanchor='top', yref="y domain", showarrow=False,
                           y=1.1, text='Control flags', row=7, col=1)

        #######
    # journal fail
        fig.add_traces(px.timeline(journalfail_df, x_start='ddatebegin', x_end='ddateend',
                       y='group_num', color='group_num')['data'], rows=8, cols=1)
        fig.add_annotation(xanchor='auto', xref="x domain", yanchor='top', yref="y domain", showarrow=False,
                           y=1.15, text='Journal failures', row=8, col=1)

        #######
    # disorders
        fig.add_traces(px.timeline(models_preds, x_start="p_begin", x_end="p_end",
                      y='model_type', color='fault_type_int',
                      color_discrete_map={'-1': 'LightSkyBlue',
                                          '0': 'MintCream',
                                          '1': 'LightPink'})['data'],
                       rows=9, cols=1)
        fig.add_annotation(xanchor='auto', xref="x domain", yanchor='top', yref="y domain", showarrow=False,
                           y=1.2, text='Models predictions', row=9, col=1)
    
        #######
        
        fig.update_layout({'xaxis1': {'type': 'date', 'showticklabels': True},
                           'xaxis2': {'type': 'date'},
                           'xaxis3': {'type': 'date'},
                           'xaxis4': {'type': 'date', 'showticklabels': True},
                           'xaxis5': {'type': 'date', 'showticklabels': True},
                           'xaxis6': {'type': 'date', 'showticklabels': True},
                           'xaxis7': {'type': 'date', 'showticklabels': True},
                           'xaxis8': {'type': 'date', 'showticklabels': True},
                           'xaxis9': {'type': 'date'}}, barmode='overlay',
                           margin_pad=0, margin_t=20, margin_b=10, coloraxis_showscale=False,
                           showlegend=False)
        print(fig)
        return fig

    #######################################################
    
        ### ETL
    
    melts_df = db.query('SELECT p_id, p_begin, p_end, p_s2_begin, p_s2_end\
                   FROM public.report1\
                   WHERE p_end >= %s AND p_end < %s AND furnace_id = 12\
                   ORDER BY p_begin',  (start_date, end_date))

    melts_df['y_order'] = 'melts'
    #
    models_preds = db.query('SELECT r1.p_begin, r1.p_end, r2.datetime_gen,\
                    r2.p_id, r2.model_type, r2.fault_type_int,\
                    r2.additive_weight, r2.additive_type\
                   FROM public.report2 r2\
                   JOIN public.report1 r1\
                   ON r1.p_id = r2.p_id\
                   WHERE r1.p_end >= %s AND r1.p_end < %s AND r1.furnace_id = 12\
                   ORDER BY r2.datetime_gen',  (start_date, end_date))

    models_preds['fault_type_int'] = models_preds['fault_type_int'].astype('str')
    #
    failures_df = db.query('SELECT dt_begin, dt_end, event_type\
                           FROM public.reportstop rs\
                           WHERE rs.dt_end >= %s AND rs.dt_end < %s\
                           ORDER BY rs.dt_begin', (start_date, end_date))

    if failures_df.empty:
        failures_df = pd.DataFrame(columns=['dt_begin', 'dt_end', 'event_type'])
        #print(failures_df)
           
    #
    try:
        journalfail_df = db.query('SELECT ipf.nreasonstayid, ipf.ddatebegin,\
                        ipf.ddateend, ipf.ntype, rst.reason_stay_name\
                       FROM public.azf1_idlepowerfurnaces ipf\
                       JOIN public.reason_stay rst\
                       ON ipf.nreasonstayid = rst.reason_stay_id\
                       WHERE ddateend >= %s AND ddateend < %s\
                       ORDER BY ddatebegin', (start_date, end_date))
    

        journalfail_df = journalfail_df.merge(failreason_map, how='left', left_on='reason_stay_name', right_on='Name')
        journalfail_df['group_num'] = journalfail_df['group_num'].fillna(99).astype('int').astype('str')
        journalfail_df['group_str'] = journalfail_df['group_str'].fillna('Прочее')

    except:
        journalfail_df = pd.DataFrame(columns=['ddatebegin', 'ddateend', 'group_num', 'group_str'])
        #
    features_df = db.query('SELECT *\
                   FROM public.bts b\
                   WHERE timestamp >= %s AND timestamp < %s\
                   ORDER BY timestamp', (start_date, end_date))

        ### Processing
    ## 
    win_len = thres[0]
    i_ab_thres = thres[1]
    i_bc_thres = thres[2]
    i_ac_thres = thres[3]
    perem_ab_thres = thres[4]
    perem_bc_thres = thres[5]
    perem_ac_thres = thres[6]
    i_max = thres[7]
    
    proc = RolledDifference()

    proc.get(features_df, 'i_ab', 'i_a_val0', 'i_b_val0', win_len, i_ab_thres)
    proc.get(features_df, 'i_bc', 'i_b_val0', 'i_c_val0', win_len, i_bc_thres)
    proc.get(features_df, 'i_ac', 'i_a_val0', 'i_c_val0', win_len, i_ac_thres)
    proc.get(features_df, 'perem_ab', 'perem_a_val0', 'perem_b_val0', win_len, perem_ab_thres)
    proc.get(features_df, 'perem_bc', 'perem_b_val0', 'perem_c_val0', win_len, perem_bc_thres)
    proc.get(features_df, 'perem_ac', 'perem_a_val0', 'perem_c_val0', win_len, perem_ac_thres)

    features_df['i_a_max_flag'] = features_df['i_a_val0'] > i_max
    features_df['i_b_max_flag'] = features_df['i_b_val0'] > i_max
    features_df['i_c_max_flag'] = features_df['i_c_val0'] > i_max

    trans = FailPeriods()

    infrastr = trans.transform_all(features_df, ['i_ab_flag', 'i_bc_flag', 'i_ac_flag',
                                                 'perem_ab_flag', 'perem_bc_flag', 'perem_ac_flag',
                                                'i_a_max_flag', 'i_b_max_flag', 'i_c_max_flag'])

    for col in ['auto_prm1_val0', 'auto_prm2_val0', 'auto_prm3_val0', 'minstralp62psn_on_val0', 'upk']:
        features_df[f'{col}_inv'] = (features_df[col] == 0).astype('int64')
    
    control_flags = trans.transform_all(features_df, ['auto_prm1_val0_inv', 'auto_prm2_val0_inv', 'auto_prm3_val0_inv',
                                                      'minstralp62psn_on_val0_inv', 'upk_inv', 'minstralblock_hoist_val0'])
    #print(infrastr.head(5))
    
    ##
    fig = common_chart(feature1, feature2, feature3)
    return fig

app.run()

Processing column i_ab_flag
Column processed, df shape: (7, 4)
Processing column i_bc_flag
Column processed, df shape: (25, 4)
Processing column i_ac_flag
Column processed, df shape: (1, 4)
Processing column perem_ab_flag
Column processed, df shape: (21, 4)
Processing column perem_bc_flag
Column processed, df shape: (3, 4)
Processing column perem_ac_flag
Column processed, df shape: (13, 4)
Processing column i_a_max_flag
Column processed, df shape: (50, 4)
Processing column i_b_max_flag
Column processed, df shape: (44, 4)
Processing column i_c_max_flag
Column processed, df shape: (144, 4)
All columns processed successfully, total df shape: (308, 5)
Processing column auto_prm1_val0_inv
Column processed, df shape: (11, 4)
Processing column auto_prm2_val0_inv
Column processed, df shape: (9, 4)
Processing column auto_prm3_val0_inv
Column processed, df shape: (4, 4)
Processing column minstralp62psn_on_val0_inv
Column processed, df shape: (61, 4)
Processing column upk_inv
Column processed, df

[2023-10-23 16:03:21,540] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Program Files\Python311\Lib\site-packages\pandas\core\indexes\base.py", line 3802, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "pandas\_libs\index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 165, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 5745, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 5753, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'fault_type_int'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\Nikolay.Kadilnikov\AppData\Roaming\Python\Python311\site-packages\flask\app.py", line 2529, in wsgi_app
    response = self.full_dispatch_re

Processing column i_ab_flag
Column processed, df shape: (3, 4)
Processing column i_bc_flag
Column processed, df shape: (1, 4)
Processing column i_ac_flag
Column processed, df is empty!!!
Processing column perem_ab_flag
Column processed, df shape: (8, 4)
Processing column perem_bc_flag
Column processed, df shape: (1, 4)
Processing column perem_ac_flag
Column processed, df shape: (2, 4)
Processing column i_a_max_flag
Column processed, df shape: (139, 4)
Processing column i_b_max_flag
Column processed, df shape: (26, 4)
Processing column i_c_max_flag
Column processed, df shape: (82, 4)
All columns processed successfully, total df shape: (262, 5)
Processing column auto_prm1_val0_inv
Column processed, df shape: (4, 4)
Processing column auto_prm2_val0_inv
Column processed, df shape: (4, 4)
Processing column auto_prm3_val0_inv
Column processed, df is empty!!!
Processing column minstralp62psn_on_val0_inv
Column processed, df shape: (41, 4)
Processing column upk_inv
Column processed, df shape: 

[2023-10-24 16:58:54,059] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Program Files\Python311\Lib\site-packages\pandas\core\indexes\base.py", line 3802, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "pandas\_libs\index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 165, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 5745, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 5753, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'fault_type_int'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\Nikolay.Kadilnikov\AppData\Roaming\Python\Python311\site-packages\flask\app.py", line 2529, in wsgi_app
    response = self.full_dispatch_re

Processing column i_ab_flag
Column processed, df shape: (3, 4)
Processing column i_bc_flag
Column processed, df shape: (1, 4)
Processing column i_ac_flag
Column processed, df is empty!!!
Processing column perem_ab_flag
Column processed, df shape: (8, 4)
Processing column perem_bc_flag
Column processed, df shape: (1, 4)
Processing column perem_ac_flag
Column processed, df shape: (2, 4)
Processing column i_a_max_flag
Column processed, df shape: (185, 4)
Processing column i_b_max_flag
Column processed, df shape: (39, 4)
Processing column i_c_max_flag
Column processed, df shape: (89, 4)
All columns processed successfully, total df shape: (328, 5)
Processing column auto_prm1_val0_inv
Column processed, df shape: (5, 4)
Processing column auto_prm2_val0_inv
Column processed, df shape: (4, 4)
Processing column auto_prm3_val0_inv
Column processed, df is empty!!!
Processing column minstralp62psn_on_val0_inv
Column processed, df shape: (52, 4)
Processing column upk_inv
Column processed, df shape: 

[2023-10-24 16:59:12,900] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Nikolay.Kadilnikov\AppData\Roaming\Python\Python311\site-packages\flask\app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Nikolay.Kadilnikov\AppData\Roaming\Python\Python311\site-packages\flask\app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Nikolay.Kadilnikov\AppData\Roaming\Python\Python311\site-packages\flask\app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Nikolay.Kadilnikov\AppData\Roaming\Python\Python311\site-packages\flask\app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

Processing column i_ab_flag
Column processed, df is empty!!!
Processing column i_bc_flag
Column processed, df is empty!!!
Processing column i_ac_flag
Column processed, df is empty!!!
Processing column perem_ab_flag


[2023-10-24 16:59:14,475] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Program Files\Python311\Lib\site-packages\pandas\core\indexes\base.py", line 3802, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "pandas\_libs\index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 165, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 5745, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 5753, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'fault_type_int'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\Nikolay.Kadilnikov\AppData\Roaming\Python\Python311\site-packages\flask\app.py", line 2529, in wsgi_app
    response = self.full_dispatch_re

Processing column i_ab_flag
Column processed, df shape: (4, 4)
Processing column i_bc_flag
Column processed, df shape: (2, 4)
Processing column i_ac_flag
Column processed, df shape: (2, 4)
Processing column perem_ab_flag
Column processed, df shape: (8, 4)
Processing column perem_bc_flag
Column processed, df shape: (2, 4)
Processing column perem_ac_flag
Column processed, df shape: (2, 4)
Processing column i_a_max_flag
Column processed, df shape: (230, 4)
Processing column i_b_max_flag
Column processed, df shape: (58, 4)
Processing column i_c_max_flag
Column processed, df shape: (49, 4)
All columns processed successfully, total df shape: (357, 5)
Processing column auto_prm1_val0_inv
Column processed, df shape: (1, 4)
Processing column auto_prm2_val0_inv
Column processed, df shape: (2, 4)
Processing column auto_prm3_val0_inv
Column processed, df is empty!!!
Processing column minstralp62psn_on_val0_inv
Column processed, df shape: (55, 4)
Processing column upk_inv
Column processed, df shape